<a href="https://colab.research.google.com/github/JP-SystemsX/24cast/blob/master/AI_Grid_Hackathon_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q wget

In [ ]:
!pip install -q autogluon

In [ ]:
!pip install --upgrade pyarrow

In [ ]:
!ls

In [ ]:
!wget https://zenodo.org/records/3899018/files/LoadProfile_20IPs_2016.csv -q
!wget https://zenodo.org/records/3899018/files/LoadProfile_30IPs_2017.csv -q


In [ ]:
import pandas as pd

#lp_20ips_2016 = pd.read_csv("LoadProfile_20IPs_2016.csv", sep=";", skiprows=1)
#lp_30ups_2017 = pd.read_csv("LoadProfile_30IPs_2017.csv", sep=";", skiprows=1)
train_2016 = pd.read_csv("tune/2016_train.csv")
train_2017 = pd.read_csv("tune/2017_train.csv")
val_2016 = pd.read_csv("tune/2016_val.csv")
val_2017 = pd.read_csv("tune/2017_val.csv")

In [ ]:
lp_20ips_2016.head()

In [ ]:
lp_30ups_2017.head()

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
df = lp_20ips_2016[["Time stamp", "LG 1"]]
df.loc[:, ["id"]] = "LG 01"

df.loc[:, ["Time stamp"]] = df["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
df.loc[:, ["Time stamp"]] = pd.to_datetime(df["Time stamp"], format='%d.%m.%Y %H:%M:%S')

In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="id",
    timestamp_column="Time stamp"
)
train_data

In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="Time stamp"
)
train_data

In [ ]:

%%time

predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MASE",
    freq="60min"
)

# Dev Setting
predictor = predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=5*60,
)

In [ ]:

%%time
# TODO: Smarter Train-Test Split
test = deepcopy(lp_30ups_2017)#[["Time stamp", "LG 1"]]
test.loc[:, ["Time stamp"]] = test["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
test.loc[:, ["Time stamp"]] = pd.to_datetime(test["Time stamp"], format='%d.%m.%Y %H:%M:%S')

# TODO: Check whether makes sense (AG can't handle Nans)
test = test.interpolate(method="linear")

# Concat Columns & Assign Ids
test = test.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
test_data = TimeSeriesDataFrame.from_data_frame(
    test.reset_index(drop=True),
    id_column="item_id",
    timestamp_column="Time stamp"
)
predictor.evaluate(test_data)

In [ ]:
predictor.leaderboard()